<a href="https://colab.research.google.com/github/virtual-bug/GP/blob/master/RecommenditionLogRegModel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np 
import pandas as pd 

import os

In [0]:
original_df =  pd.read_csv('Speed Data.csv', encoding="ISO-8859-1")

original_df.head()

,iid,id,gender,idg,condtn,wave,round,position,positin1,order,...,attr3_3,sinc3_3,intel3_3,fun3_3,amb3_3,attr5_3,sinc5_3,intel5_3,fun5_3,amb5_3
0,1,1.0,0,1,1,1,10,7,NaN,4,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
1,1,1.0,0,1,1,1,10,7,NaN,3,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
2,1,1.0,0,1,1,1,10,7,NaN,10,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
3,1,1.0,0,1,1,1,10,7,NaN,5,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN
4,1,1.0,0,1,1,1,10,7,NaN,7,...,5.0,7.0,7.0,7.0,7.0,NaN,NaN,NaN,NaN,NaN


In [0]:
original_df.size


1633710

In [0]:
# droping fields we dont want  more realistic for us plus that we will not have it in production 
gj_df = original_df.drop(['id', 'gender', 'idg', 'condtn', 'wave', 'round', 'position', 'positin1', 'order', 'partner', 'int_corr', 'samerace', 'age_o', 'race_o', 'pf_o_att','pf_o_sin','pf_o_int','pf_o_fun','pf_o_amb','pf_o_sha','dec_o','attr_o','sinc_o','intel_o','fun_o','amb_o','shar_o','like_o','prob_o','met_o', 'field', 'field_cd', 'mn_sat','undergra', 'income', 'tuition', 'race', 'imprace', 'imprelig', 'zipcode', 'goal','date', 'go_out', 'career_c'], axis=1)

#apply
gj_df = gj_df.loc[:, :'dining']


gj_df.head()

,iid,pid,match,age,from,career,sports,tvsports,exercise,dining
0,1,11.0,0,21.0,Chicago,lawyer,9.0,2.0,8.0,9.0
1,1,12.0,0,21.0,Chicago,lawyer,9.0,2.0,8.0,9.0
2,1,13.0,1,21.0,Chicago,lawyer,9.0,2.0,8.0,9.0
3,1,14.0,1,21.0,Chicago,lawyer,9.0,2.0,8.0,9.0
4,1,15.0,1,21.0,Chicago,lawyer,9.0,2.0,8.0,9.0


In [0]:
# remove NaNs
gj_df = gj_df.fillna(0)

# convert number fields to integers
convert_to_int_ignore = ['id', 'match', 'from', 'income','career']
for column in gj_df:  
    if (column not in convert_to_int_ignore):
        gj_df[column] = gj_df[column].astype(int)
        
# convert 'rating' fields to categoric
def get_category(val, num_cats):
    return val % num_cats
    
def rating_to_categoric(column, num_cats):
    return column.apply(lambda row_val: rating_to_categoric(row_val, num_cats))

# give one column 5 categories 
gj_df['sports'] = gj_df['sports'].apply(lambda value: get_category(value, 5))
gj_df['tvsports'] = gj_df['tvsports'].apply(lambda value: get_category(value, 4))
gj_df['exercise'] = gj_df['exercise'].apply(lambda value: get_category(value, 4))
gj_df['dining'] = gj_df['dining'].apply(lambda value: get_category(value, 4))
        
gj_df.head()

,iid,pid,match,age,from,career,sports,tvsports,exercise,dining
0,1,11,0,21,Chicago,lawyer,4,2,0,1
1,1,12,0,21,Chicago,lawyer,4,2,0,1
2,1,13,1,21,Chicago,lawyer,4,2,0,1
3,1,14,1,21,Chicago,lawyer,4,2,0,1
4,1,15,1,21,Chicago,lawyer,4,2,0,1


In [0]:
# rename the columns for our uses
gj_df.rename(columns={'iid': 'uid', 'from': 'cityID','sports': 'gameID', 'tvsports': 'competitive', 'exercise': 'personality', 'dining': 'skill'}, inplace=True)

gj_df.head()

,uid,pid,match,age,cityID,career,gameID,competitive,personality,skill
0,1,11,0,21,Chicago,lawyer,4,2,0,1
1,1,12,0,21,Chicago,lawyer,4,2,0,1
2,1,13,1,21,Chicago,lawyer,4,2,0,1
3,1,14,1,21,Chicago,lawyer,4,2,0,1
4,1,15,1,21,Chicago,lawyer,4,2,0,1


In [0]:
# create mock matches df for our matches db

matches_db_df = gj_df.loc[:, :'match']

matches_db_df.head()


,uid,pid,match
0,1,11,0
1,1,12,0
2,1,13,1
3,1,14,1
4,1,15,1


In [0]:
#a mock user df , what we will have on the front end isa

user_df = gj_df.drop(['pid', 'match'], axis=1)
user_df.drop_duplicates('uid', inplace=True)
user_df.reset_index(drop=True, inplace=True)

user_df.head()

,uid,age,cityID,career,gameID,competitive,personality,skill
0,1,21,Chicago,lawyer,4,2,0,1
1,2,24,Alabama,law,3,2,3,2
2,3,25,Connecticut,Economist,3,0,3,0
3,4,23,Texas,lawyer/policy work,1,1,2,3
4,5,21,Bowdoin College,lawyer,2,0,3,3


In [0]:
# assuming we have just 2 locations to start (so each user should have around a min of 5 tested matches), 
# set locationIDs randomly...
import random
user_df['cityID'] = user_df['cityID'].apply(lambda value: random.randint(1,2))
#set career randomly
user_df['career'] = user_df['career'].apply(lambda value: random.randint(1,3))


In [0]:
# final user db for seeding

user_df.head()

,uid,age,cityID,career,gameID,competitive,personality,skill
0,1,21,2,2,4,2,0,1
1,2,24,2,2,3,2,3,2
2,3,25,1,1,3,0,3,0
3,4,23,1,1,1,1,2,3
4,5,21,1,1,2,0,3,3


In [0]:
# and the final matches db

matches_db_df.head(15)

,uid,pid,match
0,1,11,0
1,1,12,0
2,1,13,1
3,1,14,1
4,1,15,1
5,1,16,0
6,1,17,0
7,1,18,0
8,1,19,1
9,1,20,0


In [0]:
# exportto seed the databases

matches_db_df.to_csv('matches_db_seed.csv')
user_df.to_csv('user_db_seed.csv')

In [0]:
# add a basic 'filter' matching algorithm 

# define parameters  
age_range = 10  #we can swap the 10 with paramter from user
matching_interests = 3

def is_basic_match(user1, user2):
    
    # location match
    if user1['cityID'] != user2['cityID']:
        return False
    
    # age match
    if abs(user1['age'] - user2['age']) > age_range:
        return False
    #career match
    if user1['career'] != user2['career']:
      return False
    # interests match
    match_count = 0
    for cat_column in user1['gameID':].index:
        if user1[cat_column] == user2[cat_column]:
            match_count += 1
            
    if match_count < matching_interests:
        return False
    
    return True

def get_basic_matches_for_user_by_id(user_id):
    user1 = user_df.loc[user_id]
    match_mask = user_df.apply(lambda user2: is_basic_match(user1, user2), axis=1)
    return user_df[match_mask]

get_basic_matches_for_user_by_id(2)

,uid,age,cityID,career,gameID,competitive,personality,skill
2,3,25,1,1,3,0,3,0
160,162,27,1,1,3,3,3,0
176,178,35,1,1,3,0,3,1
199,201,25,1,1,3,0,0,0


In [0]:
# let's create a mockup 'matches' df from the gj_df now we have users to match
# and add some extra featutes

matches_df = matches_db_df

In [0]:
# create matches data frame from the iid, pid and match rating of the users
matches_df = matches_df.pivot(index='pid', columns='uid', values='match')

# preview first 20 users vs matches 
matches_df.loc[0:20, 0:20]

uid,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20
pid,,,,,,,,,,,,,,,,,,,,
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0


In [0]:
# transform our categorical variables to dummy variables

user_df_dummies = pd.get_dummies(
    user_df.loc[:, 'gameID':], 
    prefix=['gameID','comp','pers','skill'],
    columns=['gameID', 'competitive', 'personality', 'skill']
)

user_df_dummies.head(10)

,gameID_0,gameID_1,gameID_2,gameID_3,gameID_4,comp_0,comp_1,comp_2,comp_3,pers_0,pers_1,pers_2,pers_3,skill_0,skill_1,skill_2,skill_3
0,0,0,0,0,1,0,0,1,0,1,0,0,0,0,1,0,0
1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0
2,0,0,0,1,0,1,0,0,0,0,0,0,1,1,0,0,0
3,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1
4,0,0,1,0,0,1,0,0,0,0,0,0,1,0,0,0,1
5,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,1
6,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0
7,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0
8,0,0,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0
9,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,1


In [0]:
# replace categoric columns with dummy vars

user_df = user_df.drop(['gameID', 'competitive', 'personality', 'skill'], axis=1)
user_df = user_df.join(user_df_dummies)

user_df.head()

ValueError: ignored

In [0]:
# now append the user attributes for each pid

train_df = user_df.join(matches_df, on='uid')

train_df.head()

ValueError: ignored

In [0]:
# rearrange columns moving user df columns to the end

move_to_end = user_df.columns.tolist()
move_to_end.remove('uid')

cols = train_df.columns.tolist()
cols = [col for col in cols if col not in move_to_end]
cols = cols + move_to_end
train_df = train_df[cols]

# and remove non-categoric variables for now, we'll just assume we had enough data to filter on these
train_df = train_df.drop(['age','cityID','career'], axis=1)

train_df.head(10)


,uid,1,2,3,4,5,6,7,8,9,...,comp_2,comp_3,pers_0,pers_1,pers_2,pers_3,skill_0,skill_1,skill_2,skill_3
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,1,0,0,0,0,1,0,0
1,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,0,0,1,0,0,1,0
2,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,1,0,0,0
3,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,1,0,0,0,0,1
4,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,1,0,0,0,1
5,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,1,0,0,0,0,0,1
6,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,1,1,0,0,0,0,0,1,0
7,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,1,0,0,0,0,1,0
8,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,1,0,1,0,0,1,0,0,0
9,10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,1,0,0,0,0,0,1


In [0]:
def get_train_df_for_user_by_id(id):
    train_cols = train_df.columns.tolist()
    user_cols = user_df.columns.tolist()
    cols = [col for col in train_cols if col in user_cols or col == id]
    user_train_df = train_df[cols]
    user_train_df = user_train_df.dropna()
    user_train_df = user_train_df.rename(columns={id: 'match'})
    user_train_df = user_train_df.set_index('uid')
    return user_train_df

In [0]:
# this is what we end up with to train a model for user with iid 206
# 206 is a nice example because we have 20 reted matches with a good mix of 1 and 0

get_train_df_for_user_by_id(206)

,match,gameID_0,gameID_1,gameID_2,gameID_3,gameID_4,comp_0,comp_1,comp_2,comp_3,pers_0,pers_1,pers_2,pers_3,skill_0,skill_1,skill_2,skill_3
uid,,,,,,,,,,,,,,,,,,
214,0.0,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,0
215,0.0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,1,0,0
216,0.0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,0,1
217,0.0,0,1,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1
218,1.0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,1
219,0.0,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1
220,1.0,0,0,0,0,1,0,1,0,0,0,1,0,0,0,0,1,0
221,1.0,1,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1
222,0.0,1,0,0,0,0,0,1,0,0,0,0,1,0,1,0,0,0


In [0]:
# train a logistic regression model 
from sklearn import linear_model
from sklearn import model_selection
from sklearn import metrics


user_train_df = get_train_df_for_user_by_id(208)
Y = user_train_df.loc[:, 'match']
X = user_train_df.drop(['match'], axis=1)
  
user_train_X, user_test_X, user_train_Y, user_test_Y = model_selection.train_test_split(X, Y, test_size=0.5)
def tr():
  logistic = linear_model.LogisticRegression()
  logistic.fit(user_train_X, user_train_Y)

  predictions = logistic.predict(user_test_X)
#print(predictions)
#print(user_test_Y)
  return(predictions,user_test_Y,metrics.accuracy_score(user_test_Y, predictions))
  
  
tr()

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(array([1., 0., 0., 1., 1., 0., 0., 1.]), uid
 229    1.0
 216    0.0
 222    0.0
 217    1.0
 223    0.0
 220    1.0
 232    0.0
 233    1.0
 Name: match, dtype: float64, 0.75)

In [0]:
tr()


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


(array([1., 0., 0., 1., 1., 0., 0., 1.]), uid
 229    1.0
 216    0.0
 222    0.0
 217    1.0
 223    0.0
 220    1.0
 232    0.0
 233    1.0
 Name: match, dtype: float64, 0.75)

In [0]:
matches_db_seed =  pd.read_csv('matches_db_seed.csv', encoding="ISO-8859-1")

matches_db_seed.head(20)

,Unnamed: 0,uid,pid,match
0,0,1,11,0
1,1,1,12,0
2,2,1,13,1
3,3,1,14,1
4,4,1,15,1
5,5,1,16,0
6,6,1,17,0
7,7,1,18,0
8,8,1,19,1
9,9,1,20,0


In [0]:
user_db_seed = pd.read_csv('user_db_seed.csv')
user_db_seed.head()

,Unnamed: 0,uid,age,cityID,career,gameID,competitive,personality,skill
0,0,1,21,2,2,4,2,0,1
1,1,2,24,2,2,3,2,3,2
2,2,3,25,1,1,3,0,3,0
3,3,4,23,1,1,1,1,2,3
4,4,5,21,1,1,2,0,3,3
